In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from string import digits
import sys
import html
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_excel('SampleInput.xlsx')

In [3]:
l=list()
for c,i in enumerate(df.Title):
    if (str(i).find('Ticket Assigned to')!=-1) or (str(i).find('Ticket Updated')!=-1) or (str(i).find('Ticket Marked Overdue')!=-1) or (str(i).find('Status Changed')!=-1):
        l.append(c)
        #df_new=df.drop(df.index[c])
    if str(i).find('nan')!=-1:
        l.append(c)

In [4]:
df_new=df.drop(df.index[l])

In [5]:
len(df_new)

109

In [6]:
import textract

In [7]:
# I placed few question marks in the word document in last few headings and named the changed file with addition of QM
text = textract.process("SampleDocuments/SampleInputDoc1-FAQs_QM.docx").decode("utf-8") 

In [8]:
list_input_doc1=text.split('\n\n')

In [9]:
df_faq=pd.DataFrame(columns=['Problem','Solution'])
c=-1
for i in list_input_doc1:
    try:
        if i.find('?')!=-1:
            c+=1
            df_faq.loc[c,'Problem']=i
        else:
            df_faq.loc[c,'Solution']=str(df_faq.loc[c,'Solution'])+ '\n'+i
            df_faq.loc[c,'Solution']=df_faq.loc[c,'Solution'].replace('nan\n','')
    except:
        pass

In [10]:
text_input_doc2 = textract.process("SampleDocuments/SampleInputDoc2-percent.docx").decode("utf-8") 

In [11]:
list_input_doc2=text_input_doc2.split('\n\n')

In [12]:
df_input_doc_2=pd.DataFrame(columns=['Problem','Solution'])
c=-1        
for i in list_input_doc2:
    try:
        if i.find('%')!=-1:
            c+=1
            df_input_doc_2.loc[c,'Problem']=i.replace('%','')
        else:
            df_input_doc_2.loc[c,'Solution']=str(df_input_doc_2.loc[c,'Solution'])+ '\n'+i
            df_input_doc_2.loc[c,'Solution']=df_input_doc_2.loc[c,'Solution'].replace('nan\n','')
    except:
        pass

In [13]:
import docx

In [14]:
doc = docx.Document('SampleDocuments/SampleInputDoc3-Hardware Problems.docx')

In [15]:
c=-1
df_hardware=pd.DataFrame(columns=['Main_Problem','Symptom','Diagnosis'])
for i in range(len(doc.paragraphs)):
    if(str(doc.paragraphs[i].style).find('Heading 4')!=-1):
        Main_Problem=doc.paragraphs[i].text
    elif(str(doc.paragraphs[i].style).find('Heading 5')!=-1):
        if(doc.paragraphs[i].text=='Symptom'):
            c+=1
            df_hardware.loc[c,'Main_Problem']=Main_Problem
            df_hardware.loc[c,'Symptom']=doc.paragraphs[i+1].text
        if(doc.paragraphs[i].text=='Diagnosis'):
            j=i
            try:
                while((doc.paragraphs[j+1].text!='Symptom') | (str(doc.paragraphs[j+1].style).find('Heading 4')!=-1)):
                    df_hardware.loc[c,'Diagnosis']=str(df_hardware.loc[c,'Diagnosis'])+'\n'+doc.paragraphs[j].text
                    df_hardware.loc[c,'Diagnosis']=df_hardware.loc[c,'Diagnosis'].replace('nan\nDiagnosis','')
                    j+=1
            except:
                pass
            i=j

In [16]:
df_hardware.loc[10,'Diagnosis']='''Programs that require network drives to run or operate properly: SIS, HR, FRS, PROD – ALPHA, Network Shares, and some school applications. You would also need a network connection to print to the network laser or color laser printers within CCRI.'''
df_hardware.loc[8,'Diagnosis']='Reboot the computer and see if that corrects the problem. If not check to see if there is insufficient memory.'
df_hardware.loc[11,'Diagnosis']='SampleDocuments/pic_references/11.png'
df_hardware.loc[12,'Diagnosis']='SampleDocuments/pic_references/12.png'
df_hardware.loc[15,'Diagnosis']='SampleDocuments/pic_references/15.png'
df_hardware.loc[18,'Diagnosis']='SampleDocuments/pic_references/18.png'

In [17]:
def text_cleaning(text, escape_list=[], stop=[]):
    l=[]
    """
    Text cleaning function:
        Input: 
            -text: a string variable, the text to be cleaned
            -escape_list : words not to transform by the cleaning process (only lowcase transformation is needed)  
            -stop : custom stopwords
        Output:
            -text cleaned and stemmed           
    """
    
    """ Get stop word list from package"""
    StopWords = list(set(stopwords.words('english')))
    custom_stop = StopWords + stop
    
    """ Step 1: Parse html entities"""
    text = html.unescape(text)
    text=text.replace('\n',' ').replace('\t',' ').replace('’','')
    
    tokenz=nltk.word_tokenize(text)
       
    """ Step 2: Remove stop words """
    tokenz=([token for token in tokenz if token not in custom_stop]) 

    """ Step 4: Remove digits from tokens"""
    remove_digits = str.maketrans('', '', digits)
    tokenz=[token.translate(remove_digits)  if token not in  escape_list else token for token in tokenz   ]
    
    """ Step 5: Lowcase the text"""
    tokenz=([token.lower() for token in tokenz])
    """ Step 6: Stemming the text"""
    tokenz=[EnglishStemmer().stem(token) if token not in escape_list else token for token in tokenz ]

    return ' '.join(tokenz)

In [18]:
df_new_4join=df_new.loc[:,['Title','Resolution']]
df_new_4join.columns=['Problem','Solution']

In [19]:
df_faq.columns=['Problem','Solution']

In [20]:
df_hardware_4join=df_hardware.loc[:,['Symptom','Diagnosis']]
df_hardware_4join.columns=['Problem','Solution']

In [21]:
df_all = pd.concat([df_new_4join,df_faq, df_hardware_4join,df_input_doc_2])

In [22]:
clean_df_all=df_all.copy()
clean_df_all['Problem']=df_all['Problem'].apply(text_cleaning)
clean_df_all.reset_index(drop=True,inplace=True)

In [23]:
import pickle
clean_df_all.to_pickle("clean_df_all.pkl")